# Part 9 -- Predictive Modeling

Prepare data for **Singular Value Decomposition (SVD)**.

### Load lib codes:

In [53]:
!pwd

/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter


In [54]:
from os import chdir
chdir('/home/jovyan/work/Portfolio/predicting_stock_market_trends_with_Twitter/')

from lib import *
from lib.twitter_keys import my_keys
# suppress_warnings()
%matplotlib inline

# DO IT ON RAW DATA --> TFIDF TO LOGISTIC REGRESSION --> GRIDSEARCH TUNE LOG REG & TRY RF, SVC, KNN.

* THEN do a GridSearch on the best one
* THEN CIRCLE BCAK AND EDA ON CLUSTERS

## Logistic Regression

In [71]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [61]:
X = joblib.load('../predicting_stock_market_trends_with_Twitter/data/X_tfidf.pickle')

In [62]:
y = joblib.load('../predicting_stock_market_trends_with_Twitter/data/y_AAPL_le.pickle')

In [63]:
X.shape

(68006, 5268)

In [27]:
y.shape

(68006,)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 42)

**Regularization: The problem of overfitting**

You don't want your hypothesis to have high bias (underfit) or take too many features and the learned hypthesis will learn the training set really well, but not generalize to new data as well (predict prices on new data). 

If you think overfitting is occurring, you can 

In [67]:
lr = LogisticRegression()
lr.fit(X, y)

print('Logreg intercept:', lr.intercept_)
print('Logreg coef(s):', lr.coef_)
print('Logreg predicted probabilities:', lr.predict_proba(X[0:5,:]))

Logreg intercept: [ 2.29679909 -2.63650072 -3.71046044]
Logreg coef(s): [[-0.1096753   0.1179682   0.11222695 ..., -0.70791925  0.25322048
   0.72242858]
 [-0.14108511 -0.23570642 -0.2716486  ...,  0.99141145  0.00489344
  -0.45665065]
 [ 0.40972848  0.0263229   0.2050937  ..., -0.24414754 -0.30774031
  -0.41885802]]
Logreg predicted probabilities: [[ 0.9544442   0.02200659  0.02354921]
 [ 0.9529399   0.03619128  0.01086882]
 [ 0.87380439  0.06853616  0.05765945]
 [ 0.92790362  0.04227912  0.02981726]
 [ 0.85095723  0.12496628  0.02407649]]


In [70]:
np.logspace(-3,3,30)

array([  1.00000000e-03,   1.61026203e-03,   2.59294380e-03,
         4.17531894e-03,   6.72335754e-03,   1.08263673e-02,
         1.74332882e-02,   2.80721620e-02,   4.52035366e-02,
         7.27895384e-02,   1.17210230e-01,   1.88739182e-01,
         3.03919538e-01,   4.89390092e-01,   7.88046282e-01,
         1.26896100e+00,   2.04335972e+00,   3.29034456e+00,
         5.29831691e+00,   8.53167852e+00,   1.37382380e+01,
         2.21221629e+01,   3.56224789e+01,   5.73615251e+01,
         9.23670857e+01,   1.48735211e+02,   2.39502662e+02,
         3.85662042e+02,   6.21016942e+02,   1.00000000e+03])

In [44]:
params = {
    'penalty':['l2','l1'],
    'C':np.logspace(-3,3,30)
}

In [45]:
gs_lr = GridSearchCV(LogisticRegression(random_state=42, n_jobs=-1), param_grid=params, n_jobs=-1,cv=StratifiedShuffleSplit(random_state=42))

In [46]:
gs_lr.fit(X,y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-10,   2.55955e-10,   6.55129e-10,   1.67683e-09,
         4.29193e-09,   1.09854e-08,   2.81177e-08,   7.19686e-08,
         1.84207e-07,   4.71487e-07,   1.20679e-06,   3.08884e-06,
         7.90604e-06,   2.02359e-05,   5.17947e-05,   1.32571e-04,
         3.3932...5e+08,   5.96362e+08,   1.52642e+09,
         3.90694e+09,   1.00000e+10]), 'penalty': ['l2', 'l1']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [47]:
gs_lr.best_estimator_

LogisticRegression(C=1e-10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [48]:
results = pd.DataFrame(gs_lr.cv_results_)
results[results['rank_test_score'] < 5].T

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
mean_fit_time,0.0794014,0.0976457,0.0747859,0.0920086,0.0725993,0.0922016,0.0728292,0.0951528,0.0728276,0.0935995,...,0.204656,0.219945,0.210953,0.23171,0.216997,0.252721,0.236479,0.295993,0.26777,0.359849
mean_score_time,0.00203691,0.00156097,0.00156841,0.0015583,0.00153017,0.0015336,0.00154958,0.00154157,0.00153618,0.00155125,...,0.00155959,0.00155807,0.00158348,0.00156245,0.00162497,0.00158782,0.00157723,0.00155711,0.00157146,0.00158606
mean_test_score,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,...,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627
mean_train_score,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,...,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627,0.898627
param_C,1e-10,1e-10,2.55955e-10,2.55955e-10,6.55129e-10,6.55129e-10,1.67683e-09,1.67683e-09,4.29193e-09,4.29193e-09,...,0.002223,0.00568987,0.00568987,0.0145635,0.0145635,0.0372759,0.0372759,0.0954095,0.0954095,0.244205
param_penalty,l2,l1,l2,l1,l2,l1,l2,l1,l2,l1,...,l1,l2,l1,l2,l1,l2,l1,l2,l1,l2
params,"{'C': 1e-10, 'penalty': 'l2'}","{'C': 1e-10, 'penalty': 'l1'}","{'C': 2.5595479227e-10, 'penalty': 'l2'}","{'C': 2.5595479227e-10, 'penalty': 'l1'}","{'C': 6.5512855686e-10, 'penalty': 'l2'}","{'C': 6.5512855686e-10, 'penalty': 'l1'}","{'C': 1.67683293681e-09, 'penalty': 'l2'}","{'C': 1.67683293681e-09, 'penalty': 'l1'}","{'C': 4.29193426013e-09, 'penalty': 'l2'}","{'C': 4.29193426013e-09, 'penalty': 'l1'}",...,"{'C': 0.00222299648253, 'penalty': 'l1'}","{'C': 0.00568986602902, 'penalty': 'l2'}","{'C': 0.00568986602902, 'penalty': 'l1'}","{'C': 0.014563484775, 'penalty': 'l2'}","{'C': 0.014563484775, 'penalty': 'l1'}","{'C': 0.0372759372031, 'penalty': 'l2'}","{'C': 0.0372759372031, 'penalty': 'l1'}","{'C': 0.095409547635, 'penalty': 'l2'}","{'C': 0.095409547635, 'penalty': 'l1'}","{'C': 0.244205309455, 'penalty': 'l2'}"
rank_test_score,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
split0_test_score,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,...,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618,0.898618
split0_train_score,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,...,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629,0.898629


In [51]:
print('score:', gs_lr.best_estimator_.score())
print('predict_proba:', gs_lr.best_estimator_.predict_proba())

TypeError: score() missing 2 required positional arguments: 'X' and 'y'